<a href="https://colab.research.google.com/github/cristianalethangads/projetoads/blob/main/projetoads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Visualização
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

# Geral
import numpy as np
import pandas as pd

# System
import os

from glob import glob
import shutil

import seaborn as sns

# Estilos do Seaborn para melhorar a estética
sns.set(style="whitegrid")
spectral = sns.color_palette("Spectral", as_cmap=True)